### Custom Define New Epochs Object

In [1]:
import numpy as np
from matplotlib import pyplot as plt
from mne import create_info,EpochsArray

### Simulate Data

In [2]:
sfreq = 1000.0
ch_names = ['SIM0001','SIM0002'] ## 命名模拟的通道名
ch_types = ['grad','grad'] ## 定义对应通道的类型
info = create_info(ch_names=ch_names,sfreq=sfreq,ch_types=ch_types)

In [3]:
info

<Info | 16 non-empty fields
    bads : list | 0 items
    ch_names : list | SIM0001, SIM0002
    chs : list | 2 items (GRAD: 2)
    comps : list | 0 items
    custom_ref_applied : bool | False
    dev_head_t : Transform | 3 items
    events : list | 0 items
    highpass : float | 0.0 Hz
    hpi_meas : list | 0 items
    hpi_results : list | 0 items
    lowpass : float | 500.0 Hz
    meas_date : NoneType | unspecified
    nchan : int | 2
    proc_history : list | 0 items
    projs : list | 0 items
    sfreq : float | 1000.0 Hz
    acq_pars : NoneType
    acq_stim : NoneType
    ctf_head_t : NoneType
    description : NoneType
    dev_ctf_t : NoneType
    dig : NoneType
    experimenter : NoneType
    file_id : NoneType
    gantry_angle : NoneType
    hpi_subsystem : NoneType
    kit_system_id : NoneType
    line_freq : NoneType
    meas_id : NoneType
    proj_id : NoneType
    proj_name : NoneType
    subject_info : NoneType
    xplotter_layout : NoneType
>

In [4]:
n_times = 1024  # Just over 1 second epochs
n_epochs = 40
seed = 42
rng = np.random.RandomState(seed)
noise = rng.randn(n_epochs, len(ch_names), n_times)

In [7]:
noise.shape

(40, 2, 1024)

In [8]:
# Add a 50 Hz sinusoidal burst to the noise and ramp it.
t = np.arange(n_times, dtype=np.float) / sfreq
signal = np.sin(np.pi * 2. * 50. * t)  # 50 Hz sinusoid signal
signal[np.logical_or(t < 0.45, t > 0.55)] = 0.  # Hard windowing
on_time = np.logical_and(t >= 0.45, t <= 0.55)
signal[on_time] *= np.hanning(on_time.sum())  # Ramping
data = noise + signal

In [13]:
signal.shape

(1024,)

In [14]:
on_time

array([False, False, False, ..., False, False, False], dtype=bool)

In [15]:
reject = dict(grad=4000)
events = np.empty((n_epochs, 3), dtype=int)
first_event_sample = 100
event_id = dict(sin50hz=1)
for k in range(n_epochs):
    events[k, :] = first_event_sample + k * n_times, 0, event_id['sin50hz']


In [16]:
event_id

{'sin50hz': 1}

In [17]:
events

array([[  100,     0,     1],
       [ 1124,     0,     1],
       [ 2148,     0,     1],
       [ 3172,     0,     1],
       [ 4196,     0,     1],
       [ 5220,     0,     1],
       [ 6244,     0,     1],
       [ 7268,     0,     1],
       [ 8292,     0,     1],
       [ 9316,     0,     1],
       [10340,     0,     1],
       [11364,     0,     1],
       [12388,     0,     1],
       [13412,     0,     1],
       [14436,     0,     1],
       [15460,     0,     1],
       [16484,     0,     1],
       [17508,     0,     1],
       [18532,     0,     1],
       [19556,     0,     1],
       [20580,     0,     1],
       [21604,     0,     1],
       [22628,     0,     1],
       [23652,     0,     1],
       [24676,     0,     1],
       [25700,     0,     1],
       [26724,     0,     1],
       [27748,     0,     1],
       [28772,     0,     1],
       [29796,     0,     1],
       [30820,     0,     1],
       [31844,     0,     1],
       [32868,     0,     1],
       [33

In [19]:
epochs = EpochsArray(data=data,info=info,events=events,event_id=event_id,reject=reject)


40 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
0 bad epochs dropped


### Reference
- https://martinos.org/mne/stable/auto_examples/time_frequency/plot_time_frequency_simulated.html#sphx-glr-auto-examples-time-frequency-plot-time-frequency-simulated-py